## Подготовка данных
Тут собирается очень сложно. Должен быть один файл с данными по клиенту и флагом дефолта по нему

In [1]:
#путь к файлу
strategy_data_path='C:\\Временные файлы\\strategy_data.csv'

In [2]:
#import всего нужного 
import os
import pandas as pd
#читаем файл с данными по клиентам
pwd = os.getcwd()
os.chdir(os.path.dirname(strategy_data_path))
strategy_data=pd.read_csv(os.path.basename(strategy_data_path), delimiter=';', decimal='.',quotechar="'", encoding='1251')
os.chdir(pwd)

C:\Users\Admin\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,3,4,10,12,14,36,44,61,62,66,67,80,84,87,95,100,105,107,110,118,122,123,126,129,130,132,136,138,139,145,146,149,155,157,174,182,189,198,201,204,205,206,207,208,214,215,216,217,224,226,227,228,231,232,233,234,241,242,244,245,247,248,249,253,256,270,272,274,275,277,280,284,286,287,288,295,302,303,304,305,308,309,312,316,325,326,330,331,332,335,338,340,351,354,357,358,364,365,366,367,368,369,373,376,377,380,382,397,409,410,411,412,414,415,419,420,423,424,426,427,430,431,443,446,468,472,477,480,481,486,493,498,504,505,508,510,511,514,515,516,517,520,521,523,525,536,539,544,550,557,567,568,571,573,576,577,578,587,592,595,597) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#читаем и пробразуем файл с инфой по дефолтам
strategy_result_path='C:\\Users\\Admin\\Desktop\\strategy_result.csv'
strategy_result=pd.read_csv(strategy_result_path, delimiter=';', decimal='.',quotechar="'", encoding='1251')
strategy_result.rename(index=str, columns={"ApplicationID": "ApplicationId", "target_90p_12mob_objective": "IsDefault"},inplace=True)
strategy_result.reset_index(inplace=True)
strategy_result=strategy_result[['ApplicationId', 'IsDefault']]

In [4]:
#Данные по клиентам с признаком дефолта (НО тк в предоставленных данных айди у заявок разные, джоиним рандомно)
#full_data=strategy_data.join(other=strategy_result, on='ApplicationId', how='left', rsuffix='_r')
full_data=strategy_data.join(other=strategy_result, rsuffix='_r')

# Анализ данных

In [19]:
%run -i datapy/analyze/recomendation.py

server=recommender()

## Поиск однофакторных правил

In [20]:
#Набор полей, по которым ищутся правила, передаётся клиентом
columns=['ClientIncomeAddit6','HPoint_MinValue','ClientJobSalary6','ClientExpenseTotal']
#Поле - флаг дефолта, передаётся клиентом
default_column='IsDefault'
result=server.RecommendRules(full_data, columns, default_column)
result

[['ClientIncomeAddit6', 44399.1328125, False, 0.00594848153551733],
 ['HPoint_MinValue', None, None, 0],
 ['ClientJobSalary6', 115605.765625, True, 0.004240458011197129],
 ['ClientExpenseTotal', 7171.39990234375, False, 0.005647352932099142]]

#### Пример вывода данных по предлагаемым правилам пользователю

In [23]:
#пример обработк полученных правил
def RuleToText(rule):
    column_name=rule[0]
    thr=rule[1]
    trend=rule[2]
    dr_delta=rule[3]
    if trend is None:
        return "Для поля {} не было найдено подходящих правил".format(column_name)
    elif trend:
        return "Рекомендуется ограничить выдачи по тем заявкам, где {} выше {}. Что позволит снизить DR на {}%".format(column_name, thr, round(dr_delta*100,2))
    else:
        return "Рекомендуется ограничить выдачи по тем заявкам, где {} ниже {}. Что позволит снизить DR на {}%".format(column_name, thr, round(dr_delta*100,2))
    
for rule in result:
    print(RuleToText(rule))

Рекомендуется ограничить выдачи по тем заявкам, где ClientIncomeAddit6 ниже 44399.1328125. Что позволит снизить DR на 0.59%
Для поля HPoint_MinValue не было найдено подходящих правил
Рекомендуется ограничить выдачи по тем заявкам, где ClientJobSalary6 выше 115605.765625. Что позволит снизить DR на 0.42%
Рекомендуется ограничить выдачи по тем заявкам, где ClientExpenseTotal ниже 7171.39990234375. Что позволит снизить DR на 0.56%


## Поиск многофакторных правил

In [74]:
%run -i datapy/analyze/recomendation.py
server=recommender()

#Набор полей, по которым ищутся правила, передаётся клиентом
columns=['ClientIncomeAddit6','HPoint_MinValue','ClientJobSalary6','ClientExpenseTotal']
#Поле - флаг дефолта, передаётся клиентом
default_column='IsDefault'
result=server.RecommendMultiRules(full_data, columns, default_column)
result

datapy/analyze/recomendation.py:40: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  


5

In [94]:
from sklearn.tree import _tree
dt.tree_.feature[5]

def _GetNodeDescription(DTree, node, feature_names):
    tree_=DTree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
        ]
    
    return tree_.threshold[1]
    
_GetNodeDescription(dt,5, data.drop([default_column], axis=1).columns)

69933.265625

In [104]:
def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
        ]
    print ("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print ("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            return node

    return recurse(0, 1)
    
a=tree_to_code(dt, data.drop([default_column], axis=1).columns)
print(a)

def tree(ClientIncomeAddit6, HPoint_MinValue, ClientJobSalary6, ClientExpenseTotal):
  if ClientJobSalary6 <= 115605.765625:
    if ClientJobSalary6 <= 69933.265625:
    else:  # if ClientJobSalary6 > 69933.265625
  else:  # if ClientJobSalary6 > 115605.765625
    if ClientExpenseTotal <= 6630.5:
    else:  # if ClientExpenseTotal > 6630.5
None


In [105]:
from sklearn.tree import _tree

def tree_to_code(tree, feature_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
        ]
    print ("def tree({}):".format(", ".join(feature_names)))

    def recurse(node, depth):
        indent = "  " * depth
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            print ("{}if {} <= {}:".format(indent, name, threshold))
            recurse(tree_.children_left[node], depth + 1)
            print ("{}else:  # if {} > {}".format(indent, name, threshold))
            recurse(tree_.children_right[node], depth + 1)
        else:
            print ("{}return {}".format(indent, tree_.value[node]))

    recurse(0, 1)
    
tree_to_code(dt, data.drop([default_column], axis=1).columns)

def tree(ClientIncomeAddit6, HPoint_MinValue, ClientJobSalary6, ClientExpenseTotal):
  if ClientJobSalary6 <= 115605.765625:
    if ClientJobSalary6 <= 69933.265625:
      return [[4667.  608.]]
    else:  # if ClientJobSalary6 > 69933.265625
      return [[4559.  549.]]
  else:  # if ClientJobSalary6 > 115605.765625
    if ClientExpenseTotal <= 6630.5:
      return [[7959. 1095.]]
    else:  # if ClientExpenseTotal > 6630.5
      return [[4319.  558.]]
